Tarea: Detectar y seguir personas y vehículos.

In [ ]:
# Importamos las librerías necesarias y cargamos los modelos pre-entrenados.
import cv2
import torch
import numpy as np
import pandas as pd
import pytesseract
from ultralytics import YOLO
from collections import deque

# Configurar Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Cambia la ruta si es necesario

# Cargar el modelo YOLO para detección y seguimiento
model_detection = YOLO('yolov8n.pt')  # Puedes cambiar a 'yolov11n.pt' si lo tienes

# Inicializar EasyOCR
import easyocr
reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())
